In [71]:
import pandas as pd
from ast import literal_eval
from collections import Counter
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import itertools

# 문자열을 리스트로 변환
def str_to_list(x):
    try:
        return literal_eval(x)
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return None

origin_data=pd.read_csv("/content/drive/MyDrive/성균관대/rob_selected_4.csv", low_memory=False)
hate_data = origin_data[origin_data['score']<=2]
like_data = origin_data[origin_data['score']>=4]
medium_data = origin_data[origin_data['score']==3]
data_list = [origin_data, hate_data, like_data, medium_data]

for idx, data in enumerate(data_list):
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]
  tfidf = TfidfVectorizer()
  tdm = tfidf.fit_transform(data['words_selected'])
  tf_idf = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    '빈도': tdm.sum(axis=0).flat})
  tf_idf = tf_idf.sort_values('빈도', ascending=False)
  tf_idf_dict = tf_idf.set_index('단어')['빈도'].to_dict()
  not_remove = dict(itertools.islice(tf_idf_dict.items(), 1000))

  count_vec = CountVectorizer()
  con = count_vec.fit_transform(data['words_selected'])
  word_counts = np.array(con.sum(axis=0)).squeeze()
  # 단어-빈도수 사전 생성
  c_dict = dict(zip(count_vec.get_feature_names_out(), word_counts))
  # 빈도수를 기준으로 내림차순 정렬된 단어-빈도수 사전 생성
  c_dict = dict(sorted(c_dict.items(), key=lambda x: x[1], reverse=True))
  count = {}   #동시출현 빈도가 저장될 dict
  for line in data['words_selected']:
      #하나의 문서에서 동일한 단어가 두번 나와도 두번의 동시출현으로 고려X
      words = line.split(" ")
      #한줄씩 읽어와서 단어별로 분리(unique한 값으로 받아오기)
      #split은 띄어쓰기를 단어로 구분하라는 함수 
      
      for i, a in enumerate(words):
          for b in words[i+1:]:
              if a!=b:
                if a>b: 
                    count[b, a] = count.get((b, a),0) + 1  
                else :
                    count[a, b] = count.get((a, b),0) + 1
  df=pd.DataFrame.from_dict(count, orient='index')
  df.head()
  list1=[]
  for i in range(len(df)):
    #index를 중심으로 계속 중첩해서 list에 넣는다 
    list1.append([df.index[i][0],df.index[i][1],df[0][i]])
      
  #pandas 이용해서 df형태로 만들기 
  df2=pd.DataFrame(list1, columns=["term1","term2","freq"])
  #pandas 이용해서 sorting 하기 (디폴트가 오름차순이라서 false 꼭 써줘야 내림차순으로 나옴)
  df3=df2.sort_values(by=['freq'],ascending=False)
  print(df3)
  import numpy as np
  import networkx as nx
  import operator
  G=nx.Graph()
  for key in c_dict:
      G.add_node(key, tfidf=tf_idf_dict[key], count=c_dict[key])
  for i in range(len(df3)):
      G.add_edge(df3['term1'][i], df3['term2'][i], weight=int(df3['freq'][i]))
  nodes_to_remove = set(G.nodes()) - set(not_remove)  # 제거하지 않을 노드들의 집합
  G.remove_nodes_from(nodes_to_remove)
  print(len(G.nodes), len(G.edges))
  if idx ==0:
      temp = "origin"
  if idx == 1:
      temp = "hate"
  if idx == 2:
      temp = "like"
  if idx == 3:
      temp = "medium"
  with open("/content/drive/MyDrive/성균관대/github/tf-idf/rob_"+temp+"_word_count.csv", 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key in c_dict:
       writer.writerow([key, c_dict[key]])
  with open("/content/drive/MyDrive/성균관대/github/tf-idf/rob_"+temp+"_tf_idf.csv", 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key in tf_idf_dict:
       writer.writerow([key, tf_idf_dict[key]])
  nx.write_graphml_lxml(G, "/content/drive/MyDrive/성균관대/github/tf-idf/rob_"+temp+"_tf_idf_graph.graphml")

              term1     term2   freq
47             game      play  92383
54              fun      game  43851
425            game    people  25526
728          friend      game  22358
320            game     thing  22228
...             ...       ...    ...
492184       server  streamer      1
492188   filtration  platform      1
492190       clever  platform      1
492191     platform    swears      1
1077658       dosen  internet      1

[1077659 rows x 3 columns]
1000 225827


<ipython-input-71-8ba1fd9a21c9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
<ipython-input-71-8ba1fd9a21c9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]


           term1      term2  freq
211         game       play  9218
481          fix       game  4412
545         game       time  3560
415         game     please  3420
303         game       star  2876
...          ...        ...   ...
141293     boleh      iaitu     1
141294  bercakap      iaitu     1
141295       dan      iaitu     1
141296     iaitu  terkeluar     1
305854     dosen   internet     1

[305855 rows x 3 columns]
1000 115317


<ipython-input-71-8ba1fd9a21c9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
<ipython-input-71-8ba1fd9a21c9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]


          term1   term2   freq
37         game    play  76797
44          fun    game  39451
522        game  people  20406
749      friend    game  20224
531        game    love  19459
...         ...     ...    ...
368920     keep     ttd      1
368921     keep    ttda      1
368922     keep   tryna      1
368923    berry     ttd      1
815814  emotion    work      1

[815815 rows x 3 columns]
1000 187985


<ipython-input-71-8ba1fd9a21c9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
<ipython-input-71-8ba1fd9a21c9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]


         term1    term2  freq
273       game     play  6368
213       game  problem  2852
455        fix     game  2809
1071      game   please  2663
4          fun     game  2603
...        ...      ...   ...
74135      guy     step     1
74134     quit     skin     1
74133   avatar     quit     1
74130   filter      try     1
158667    plan  request     1

[158668 rows x 3 columns]
1000 80819


In [72]:
import pandas as pd
from ast import literal_eval
from collections import Counter
import csv
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import itertools

# 문자열을 리스트로 변환
def str_to_list(x):
    try:
        return literal_eval(x)
    except: #해당 값이 null값이거나 오류가 있을 때, None을 return 하기
        return None

origin_data=pd.read_csv("/content/drive/MyDrive/성균관대/zep_selected_4.csv", low_memory=False)
hate_data = origin_data[origin_data['score']<=2]
like_data = origin_data[origin_data['score']>=4]
medium_data = origin_data[origin_data['score']==3]
data_list = [origin_data, hate_data, like_data, medium_data]
for idx, data in enumerate(data_list):
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]
  tfidf = TfidfVectorizer()
  tdm = tfidf.fit_transform(data['words_selected'])
  tf_idf = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    '빈도': tdm.sum(axis=0).flat})
  tf_idf = tf_idf.sort_values('빈도', ascending=False)
  tf_idf_dict = tf_idf.set_index('단어')['빈도'].to_dict()
  not_remove = dict(itertools.islice(tf_idf_dict.items(), 1000))

  count_vec = CountVectorizer()
  con = count_vec.fit_transform(data['words_selected'])
  word_counts = np.array(con.sum(axis=0)).squeeze()
  # 단어-빈도수 사전 생성
  c_dict = dict(zip(count_vec.get_feature_names_out(), word_counts))
  # 빈도수를 기준으로 내림차순 정렬된 단어-빈도수 사전 생성
  c_dict = dict(sorted(c_dict.items(), key=lambda x: x[1], reverse=True))
  count = {}   #동시출현 빈도가 저장될 dict
  for line in data['words_selected']:
      #하나의 문서에서 동일한 단어가 두번 나와도 두번의 동시출현으로 고려X
      words = line.split(" ")
      #한줄씩 읽어와서 단어별로 분리(unique한 값으로 받아오기)
      #split은 띄어쓰기를 단어로 구분하라는 함수 
      
      for i, a in enumerate(words):
          for b in words[i+1:]:
              if a!=b:
                if a>b: 
                    count[b, a] = count.get((b, a),0) + 1  
                else :
                    count[a, b] = count.get((a, b),0) + 1
  df=pd.DataFrame.from_dict(count, orient='index')
  df.head()
  list1=[]
  for i in range(len(df)):
    #index를 중심으로 계속 중첩해서 list에 넣는다 
    list1.append([df.index[i][0],df.index[i][1],df[0][i]])
      
  #pandas 이용해서 df형태로 만들기 
  df2=pd.DataFrame(list1, columns=["term1","term2","freq"])
  #pandas 이용해서 sorting 하기 (디폴트가 오름차순이라서 false 꼭 써줘야 내림차순으로 나옴)
  df3=df2.sort_values(by=['freq'],ascending=False)
  import numpy as np
  import networkx as nx
  import operator
  G=nx.Graph()
  for key in c_dict:
      G.add_node(key, tfidf=tf_idf_dict[key], count=c_dict[key])
  for i in range(len(df3)):
      G.add_edge(df3['term1'][i], df3['term2'][i], weight=int(df3['freq'][i]))
  nodes_to_remove = set(G.nodes()) - set(not_remove)  # 제거하지 않을 노드들의 집합
  G.remove_nodes_from(nodes_to_remove)
  print(len(G.nodes))
  if idx ==0:
      temp = "origin"
  if idx == 1:
      temp = "hate"
  if idx == 2:
      temp = "like"
  if idx == 3:
      temp = "medium"
  with open("/content/drive/MyDrive/성균관대/github/tf-idf/zep_"+temp+"_word_count.csv", 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key in c_dict:
       writer.writerow([key, c_dict[key]])
  with open("/content/drive/MyDrive/성균관대/github/tf-idf/zep_"+temp+"_tf_idf.csv", 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for key in tf_idf_dict:
       writer.writerow([key, tf_idf_dict[key]])
  nx.write_graphml_lxml(G, "/content/drive/MyDrive/성균관대/github/tf-idf/zep_"+temp+"_tf_idf_graph.graphml")

1000


<ipython-input-72-d60382b56452>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
<ipython-input-72-d60382b56452>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]


1000


<ipython-input-72-d60382b56452>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
<ipython-input-72-d60382b56452>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]


1000


<ipython-input-72-d60382b56452>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = data['words_selected'].apply(lambda x: str_to_list(x))
<ipython-input-72-d60382b56452>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['words_selected'] = [' '.join(row) for row in data['words_selected']]


1000
